In [134]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p

In [135]:
min_max_scaler = p.MinMaxScaler()

INPUT_DATA_PATH = 'test_input.npy'
OUTPUT_DATA_PATH = 'test_output.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

In [136]:
SPLIT_IDX = 15 #750
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

# print(x_train.shape)
# print(x_train[0].shape)
# print(x_train[0][0].shape)


# for img in x_train: # iterate through 15 images
#         for h in img[0]:
#                 for w in h:
#                         if (w < 254):
#                                 print (f"found one! = {w}")    
  
class LinesDataset(TensorDataset):
        def __init__(self, lines, labels):
                self.lines = lines
                self.labels = labels
        def __len__(self):
                return len(self.lines)
        def __getitem__(self, idx):
                line = torch.tensor(self.lines[idx])
                label = torch.tensor(self.labels[idx])
                sample = {"Line": line, "Label": label}
                return sample


train_dataset = TensorDataset(x_train,y_train) # create your datset
# train_dataloader = DataLoader(train_dataset, shuffle=True) # create your dataloader

train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

#/home/bwilab/asha_ritu/line_model/conv_network.ipynb


3


In [137]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            # first layer: learning one person. 
            # kernel size = diameter of a dot (5 pixels) + padding (2 pixels) = (7,7)
            # out_channels: 1 for now
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(7,7), stride=1, bias=True, padding=(3,3)), # input layer
            # nn.ReLU(),

            # layer 2: learning two dots.
            # kernel_size = diameter of two dots (10 pixels) + distance between them (2 pixels) + padding (2 pixels)
            # there are 8 different orientations two people can be in, resulting in 8 output channels.
            # two options: either increase kernel size and number of channels or
            # pool -> lower image resolution and learn a smaller pattern
            # nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(14,14), stride=1, bias=True, padding=(7,7)),
            # nn.ReLU(),
            # final layer: out_channels=1, 
            # kernel size = ? ask Dr. Hart what a reasonable kernel size for the output layer could be.
            # nn.Conv2d(in_channels=8, out_channels=1, kernel_size=(12,12), padding=(6,6)),
            # nn.ReLU()
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [138]:
model = NeuralNetwork()
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size = 4
epochs = 5
# Initialize the loss function
loss_fn = nn.MSELoss()
# loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
test_losses = []

In [139]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        # print(X)
        pred = model(X)
        print(f"pred:{pred}")
        print(f"target:{y}")
      

        # print(f"pred: {pred}")
        # print(f"y shape:{y.shape}")
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        
        print(f"loss: {loss:>7f}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # print(X.shape)
            pred = model(X)
            # print(pred.argmax(1).shape)
            # print(y.shape)
            test_loss += loss_fn(pred, y).item()
            correct += (pred == y).type(torch.float).sum().item()
            # print(correct)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [140]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  avg_accuracy = 0
  with torch.no_grad():
    for data, target in test_dataloader:
      # print(data.shape)
      # print(target.shape)
      output = model(data)

      # for img in output:
      #   for h in img:
      #     for w in h:
      #       if (w[0] != 0):
      #         print(f"{w[0]}")

      # print(output)
      test_loss += loss_fn(output, target).item()
      # pred = output.data.max(1, keepdim=True)[1]
      # print(pred)
      # print(target)
      
      img = 0
      h = 0
      w = 0

      while (img < 5):
        while (h < 100):
          while (w < 100):
            # print(f"target: {target[img][h][w][0]}")
            # print(f"output: {output[img][h][w][0]}")

            if ((target[img][h][w][0] != 0) and (output[img][h][w][0] != 0)):
              correct += 1
            w += 1
          h += 1
        img += 1

      avg_accuracy += correct / 1000

      # num_correct = output.eq(target.data.view_as(output)).sum()
      
  # test_loss /= len(test_dataloader.dataset)
  # test_losses.append(test_loss)
  # print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  #   test_loss, correct, len(test_dataloader.dataset),
  #   100. * correct / len(test_dataloader.dataset)))
  avg_accuracy /= 1000
  print(f"average accuracy: {avg_accuracy}")

test()

average accuracy: 1.9503000000000001


In [141]:
# test()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test()
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
pred:tensor([[[[-0.2932, -0.2689, -0.3044,  ..., -0.4976, -0.3718, -0.1765],
          [-0.1448, -0.1073, -0.2213,  ..., -0.3872, -0.3239, -0.2036],
          [-0.3199, -0.2321, -0.4247,  ..., -0.6495, -0.4903, -0.2628],
          ...,
          [-0.1605, -0.0442, -0.2436,  ..., -0.5438, -0.4693, -0.3019],
          [-0.1506, -0.0119, -0.2663,  ..., -0.4929, -0.4811, -0.3746],
          [ 0.0040,  0.0954, -0.1067,  ..., -0.2947, -0.3148, -0.2506]]],


        [[[-0.2932, -0.2689, -0.3044,  ..., -0.4976, -0.3718, -0.1765],
          [-0.1448, -0.1073, -0.2213,  ..., -0.3872, -0.3239, -0.2036],
          [-0.3199, -0.2321, -0.4247,  ..., -0.6495, -0.4903, -0.2628],
          ...,
          [-0.1605, -0.0442, -0.2436,  ..., -0.5438, -0.4693, -0.3019],
          [-0.1506, -0.0119, -0.2663,  ..., -0.4929, -0.4811, -0.3746],
          [ 0.0040,  0.0954, -0.1067,  ..., -0.2947, -0.3148, -0.2506]]],


        [[[-0.2932, -0.2689, -0.3044,  ..., -0.4976, 